In [ ]:
# Near field
# 
import meep as mp
import numpy as np

# --- Simulation parameters ---
wavelength = 0.325  # μm
frequency = 1 / wavelength
resolution = 10  # pixels/μm

pml_thickness = 1.0  # μm
padding = 2.0        # distance from source to PML
cell_dim = 2 * (padding + pml_thickness)
cell_size = mp.Vector3(cell_dim, cell_dim, cell_dim)

source_pos = mp.Vector3(0, 0, 0)
source_component = mp.Ez

def simulate_full_fields(obs_points):
    if obs_points.ndim == 1:
        obs_points = obs_points[np.newaxis, :]

    sim = mp.Simulation(
        cell_size=cell_size,
        resolution=resolution,
        boundary_layers=[mp.PML(pml_thickness)],
        sources=[mp.Source(mp.ContinuousSource(frequency=frequency),
                           component=source_component,
                           center=source_pos)]
    )

    dft_obj = sim.add_dft_fields(
        [mp.Ex, mp.Ey, mp.Ez, mp.Hx, mp.Hy, mp.Hz],
        [frequency],
        center=mp.Vector3(),
        size=cell_size
    )

    sim.run(until=100)

    Ex = sim.get_dft_array(dft_obj, mp.Ex, 0)
    Ey = sim.get_dft_array(dft_obj, mp.Ey, 0)
    Ez = sim.get_dft_array(dft_obj, mp.Ez, 0)
    Hx = sim.get_dft_array(dft_obj, mp.Hx, 0)
    Hy = sim.get_dft_array(dft_obj, mp.Hy, 0)
    Hz = sim.get_dft_array(dft_obj, mp.Hz, 0)

    grid_size = sim.get_array_metadata()  # ✅ this returns (nx, ny, nz)

    delta = 1.0 / resolution
    origin = -0.5 * np.array([cell_dim, cell_dim, cell_dim])

    def to_index(vec):
        return tuple(((vec - origin) / delta).astype(int))

    E_all = []
    H_all = []

    for pt in obs_points:
        idx = to_index(np.array(pt))
        E = np.array([Ex[idx], Ey[idx], Ez[idx]])
        H = np.array([Hx[idx], Hy[idx], Hz[idx]])
        E_all.append(E)
        H_all.append(H)

    return np.array(E_all), np.array(H_all)

# --- Example usage ---
points = np.array([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0]
])

E, H = simulate_full_fields(points)

for i, pt in enumerate(points):
    print(f"\nPoint {i} at {pt}:")
    print(f"  E = {E[i]}")
    print(f" |E| = {np.linalg.norm(E[i])}")
    print(f"  H = {H[i]}")
    print(f" |H| = {np.linalg.norm(H[i])}")
    print(f" |E|/|H| = {np.linalg.norm(E[i]) / np.linalg.norm(H[i])}")


-----------
Initializing structure...
time for choose_chunkdivision = 0.0026741 s
Working in 3D dimensions.
Computational cell is 6 x 6 x 6 with resolution 10
time for set_epsilon = 0.352193 s
-----------
Meep progress: 8.200000000000001/100.0 = 8.2% done in 4.0s, 44.8s to go
on time step 164 (time=8.2), 0.0244066 s/step
Meep progress: 19.650000000000002/100.0 = 19.7% done in 8.0s, 32.8s to go
on time step 393 (time=19.65), 0.0175108 s/step
Meep progress: 31.05/100.0 = 31.1% done in 12.0s, 26.7s to go
on time step 621 (time=31.05), 0.0175462 s/step
Meep progress: 41.300000000000004/100.0 = 41.3% done in 16.0s, 22.8s to go
on time step 827 (time=41.35), 0.0195163 s/step
Meep progress: 51.800000000000004/100.0 = 51.8% done in 20.0s, 18.6s to go
on time step 1037 (time=51.85), 0.0191098 s/step
Meep progress: 63.0/100.0 = 63.0% done in 24.0s, 14.1s to go
on time step 1261 (time=63.05), 0.0178753 s/step
Meep progress: 73.85000000000001/100.0 = 73.9% done in 28.1s, 9.9s to go
on time step 14

In [2]:
# Far field

import meep as mp
import numpy as np

# --- Simulation Parameters ---
wavelength = 0.325  # μm
frequency = 1 / wavelength
resolution = 10

pml_thickness = 1.0
padding = 2.0
cell_dim = 2 * (padding + pml_thickness)
cell_size = mp.Vector3(cell_dim, cell_dim, cell_dim)

source_pos = mp.Vector3(0, 0, 0)
source_component = mp.Ez

def simulate_far_field(far_point):
    sim = mp.Simulation(
        cell_size=cell_size,
        resolution=resolution,
        boundary_layers=[mp.PML(pml_thickness)],
        sources=[mp.Source(mp.ContinuousSource(frequency=frequency),
                           component=source_component,
                           center=source_pos)]
    )

    # Define a near-field box around the source
    nearfield_box = sim.add_near2far(
        frequency, 0, 1,  # fcen, df, nfreq
        mp.Near2FarRegion(center=mp.Vector3(), size=mp.Vector3(cell_dim - 2 * pml_thickness, 0, 0), direction=mp.X),
        mp.Near2FarRegion(center=mp.Vector3(), size=mp.Vector3(0, cell_dim - 2 * pml_thickness, 0), direction=mp.Y),
        mp.Near2FarRegion(center=mp.Vector3(), size=mp.Vector3(0, 0, cell_dim - 2 * pml_thickness), direction=mp.Z)
    )

    sim.run(until=100)

    # Calculate far-field E and H
    ff_fields = sim.get_farfield(nearfield_box, mp.Vector3(*far_point))
    E = np.array(ff_fields[0:3])
    H = np.array(ff_fields[3:6])

    return E, H

# --- Example usage ---
far_point = [10.0, 1.0, 1.0]
E_far, H_far = simulate_far_field(far_point)

print(f"\nFar-field at {far_point} μm:")
print(f"  E = {E_far}")
print(f" |E| = {np.linalg.norm(E_far)}")
print(f"  H = {H_far}")
print(f" |H| = {np.linalg.norm(H_far)}")
print(f" |E|/|H| = {np.linalg.norm(E_far) / np.linalg.norm(H_far)} (should ≈ 376.73 in vacuum)")
print("E and H are not correctly scaled")


-----------
Initializing structure...
time for choose_chunkdivision = 0.00226712 s
Working in 3D dimensions.
Computational cell is 6 x 6 x 6 with resolution 10
time for set_epsilon = 0.402315 s
-----------
Meep progress: 18.5/100.0 = 18.5% done in 4.0s, 17.7s to go
on time step 370 (time=18.5), 0.0108284 s/step
Meep progress: 40.1/100.0 = 40.1% done in 8.0s, 12.0s to go
on time step 802 (time=40.1), 0.00926762 s/step
Meep progress: 62.45/100.0 = 62.5% done in 12.0s, 7.2s to go
on time step 1249 (time=62.45), 0.00896225 s/step
Meep progress: 85.65/100.0 = 85.7% done in 16.0s, 2.7s to go
on time step 1713 (time=85.65), 0.00863472 s/step
run 0 finished at t = 100.0 (2000 timesteps)

Far-field at [10.0, 1.0, 1.0] μm:
  E = [-1.54822574 -2.81589931j -0.29552391 -1.52726061j
 15.2402359 +30.01518973j]
 |E| = 33.851477904656164
  H = [  1.72616273 +3.16242717j -15.22492201-29.99381298j
  -0.12079945 -1.22891304j]
 |H| = 33.85162434480598
 |E|/|H| = 0.999995674058405 (should ≈ 376.73 in vacuum

In [ ]:
E_far

 |E| = 3.2134548169458315


array([-1.54822574 -2.81589931j, -0.29552391 -1.52726061j,
       15.2402359 +30.01518973j])

In [3]:
# Calculating H using impedence in free space

# Compute radial direction
r_hat = far_point / np.linalg.norm(far_point)


# Impedance of free space
eta0 = 376.730

# Compute far-field H
H_far = np.cross(r_hat, E_far) / eta0

# Display results
print("E_far:", E_far)
print("H_far:", H_far)
print("|E|   =", np.linalg.norm(E_far))
print("|H|   =", np.linalg.norm(H_far))
print("|E|/|H| =", np.linalg.norm(E_far) / np.linalg.norm(H_far))


E_far: [-1.54822574 -2.81589931j -0.29552391 -1.52726061j
 15.2402359 +30.01518973j]
H_far: [ 0.00408321+0.0082902j  -0.04046235-0.07962807j -0.0003698 -0.00327396j]
|E|   = 33.851477904656164
|H|   = 0.08985591907621429
|E|/|H| = 376.7306400365668


In [4]:
import meep as mp
import numpy as np

# --- Simulation Parameters ---
wavelength = 0.325  # μm
frequency = 1 / wavelength
resolution = 10

pml_thickness = 1.0
padding = 2.0
cell_dim = 2 * (padding + pml_thickness)
cell_size = mp.Vector3(cell_dim, cell_dim, cell_dim)

source_pos = mp.Vector3(0, 0, 0)
source_component = mp.Ez

# --- Far-field Simulation Function ---
def simulate_far_field(far_point):
    sim = mp.Simulation(
        cell_size=cell_size,
        resolution=resolution,
        boundary_layers=[mp.PML(pml_thickness)],
        sources=[mp.Source(mp.ContinuousSource(frequency=frequency),
                           component=source_component,
                           center=source_pos)]
    )

    # Near-to-far field monitor surrounding the source
    nearfield_box = sim.add_near2far(
        frequency, 0, 1,  # single frequency
        mp.Near2FarRegion(center=mp.Vector3(), size=mp.Vector3(cell_dim - 2*pml_thickness, 0, 0), direction=mp.X),
        mp.Near2FarRegion(center=mp.Vector3(), size=mp.Vector3(0, cell_dim - 2*pml_thickness, 0), direction=mp.Y),
        mp.Near2FarRegion(center=mp.Vector3(), size=mp.Vector3(0, 0, cell_dim - 2*pml_thickness), direction=mp.Z)
    )

    sim.run(until=100)

    r = mp.Vector3(*far_point)
    ff_fields = sim.get_farfield(nearfield_box, r)

    # Split into E and H (6 values returned per frequency)
    ff_E = np.array(ff_fields[:3])
    ff_H = np.array(ff_fields[3:])
    
    return ff_E, ff_H

# --- Main Execution ---
if __name__ == "__main__":
    far_point = [10.0, 0.0, 0.0]  # μm
    eta0 = 376.730  # Vacuum impedance (Ohm)

    E_far, H_meep = simulate_far_field(far_point)
    r_hat = np.array(far_point) / np.linalg.norm(far_point)
    H_calc = np.cross(r_hat, E_far) / eta0

    # --- Print Results ---
    print(f"\n📍 Far-field at {far_point} μm:")
    print("E_far     =", E_far)
    print("H_meep    =", H_meep)
    print("H_calc    =", H_calc)

    print("\n--- Norms ---")
    print(f"|E|        = {np.linalg.norm(E_far):.6f}")
    print(f"|H_meep|   = {np.linalg.norm(H_meep):.6f}")
    print(f"|H_calc|   = {np.linalg.norm(H_calc):.6f}")

    print("\n--- Ratios ---")
    print(f"|E|/|H_meep|  = {np.linalg.norm(E_far)/np.linalg.norm(H_meep):.6f} (should ≈ η = {eta0})")
    print(f"|E|/|H_calc|  = {np.linalg.norm(E_far)/np.linalg.norm(H_calc):.6f} (should = η)")

    print("\n--- Difference ---")
    print(f"ΔH = ||H_meep - H_calc|| = {np.linalg.norm(H_meep - H_calc):.6e}")


-----------
Initializing structure...
time for choose_chunkdivision = 0.0010941 s
Working in 3D dimensions.
Computational cell is 6 x 6 x 6 with resolution 10
time for set_epsilon = 1.22814 s
-----------
Meep progress: 12.4/100.0 = 12.4% done in 4.0s, 28.3s to go
on time step 248 (time=12.4), 0.0161577 s/step
Meep progress: 33.45/100.0 = 33.5% done in 8.0s, 15.9s to go
on time step 670 (time=33.5), 0.00949342 s/step
Meep progress: 55.75/100.0 = 55.8% done in 12.0s, 9.5s to go
on time step 1116 (time=55.8), 0.00898299 s/step
Meep progress: 77.4/100.0 = 77.4% done in 16.0s, 4.7s to go
on time step 1549 (time=77.45), 0.00924685 s/step
Meep progress: 98.4/100.0 = 98.4% done in 20.0s, 0.3s to go
on time step 1969 (time=98.45), 0.00954078 s/step
run 0 finished at t = 100.0 (2000 timesteps)

📍 Far-field at [10.0, 0.0, 0.0] μm:
E_far     = [ 1.35470016e-01+6.86606701e-02j -4.42973837e-15-1.60798024e-15j
  1.91613003e+01-2.23774235e+01j]
H_meep    = [-6.14537156e-02-1.84313936e-01j -1.91615307e